In [1]:
# Example 2: inserting data (car_sale) in the created MySQL database (collectIGTI)
# Dataset: car_sale
# Author: Humberto Bianchini

In [2]:
# 1) Importing all necessary libraries.
import MySQLdb
import sqlalchemy
import pandas as pd
import datetime
from dateutil.relativedelta import relativedelta

In [3]:
# 2) Reading and showing the first rows of the dataset.
dataframe = pd.read_csv('datasets/car_sale.csv', sep = ';', encoding='latin-1')
dataframe.head(10)

,id_person,name,genre,date_of_birth,purchase_date,manufacturer,model,price
0,1016.0,Maria Clara,Feminino,21/11/1986,12/01/2020,Fiat,Uno Attractive 1.0,50004.0
1,1061.0,Bianca,Feminino,21/02/1989,11/02/2020,Fiat,Mobi Easy 1.5,40449.0
2,1086.0,Analu,Feminino,05/08/1977,11/02/2020,Volkswagen,Fox Connect 1.6,57640.0
3,1117.0,Samuel,Masculino,22/11/1989,11/02/2020,Fiat,Uno Attractive 1.0,50004.0
4,1139.0,Cauã,Masculino,14/09/1961,11/02/2020,Lifan,530 Talent 1.5,43990.0
5,1118.0,Enzo Gabriel,Masculino,15/02/1984,12/02/2020,Ford,Ka S 1.0,51990.0
6,1048.0,Maria,Feminino,18/09/1992,11/03/2020,Hyndai,Hyundai HB20 1.0 Sense,49590.0
7,1050.0,Ana Laura,Feminino,15/10/1997,11/03/2020,Hyndai,Hyundai HB20 1.0 Sense,49590.0
8,1075.0,Mariah,Feminino,05/11/1969,11/03/2020,Volkswagen,Gol 1.0,54150.0
9,1092.0,Ana Vitória,Feminino,05/12/1963,11/03/2020,Hyndai,Hyundai HB20 1.0 Sense,49590.0


In [4]:
# 2) Creating the database connection (MySQL)
try:
    string_connection = f"mysql://{'humberto'}:{'123456'}@{'localhost'}/{'car_sales'}"
    engine = sqlalchemy.create_engine(string_connection)
    conn = engine.connect()
    print(f"Connection Successful!")
except Exception as e:
    print(f"Connection Failed! Error: {e}")

Connection Successful!


In [5]:
# 3) Preparing data for insertion into the database
df_person = dataframe[['id_person', 'name', 'genre', 'date_of_birth']].drop_duplicates()
df_models = dataframe[['model', 'manufacturer']].drop_duplicates()
manufacturers = list(dataframe.manufacturer.unique())

In [6]:
df_person[:10]

,id_person,name,genre,date_of_birth
0,1016.0,Maria Clara,Feminino,21/11/1986
1,1061.0,Bianca,Feminino,21/02/1989
2,1086.0,Analu,Feminino,05/08/1977
3,1117.0,Samuel,Masculino,22/11/1989
4,1139.0,Cauã,Masculino,14/09/1961
5,1118.0,Enzo Gabriel,Masculino,15/02/1984
6,1048.0,Maria,Feminino,18/09/1992
7,1050.0,Ana Laura,Feminino,15/10/1997
8,1075.0,Mariah,Feminino,05/11/1969
9,1092.0,Ana Vitória,Feminino,05/12/1963


In [7]:
df_models

,model,manufacturer
0,Uno Attractive 1.0,Fiat
1,Mobi Easy 1.5,Fiat
2,Fox Connect 1.6,Volkswagen
4,530 Talent 1.5,Lifan
5,Ka S 1.0,Ford
6,Hyundai HB20 1.0 Sense,Hyndai
8,Gol 1.0,Volkswagen
10,Grand Siena Attractive 1.,Fiat
15,Kwid Life 1.0,Renault
27,Argo 1.0,Fiat


In [8]:
manufacturers

['Fiat', 'Volkswagen', 'Lifan', 'Ford', 'Hyndai', 'Renault']

In [9]:
# 3) Inserting data into the database with "intertuples" format
try: 
    for person in df_person.itertuples():
        id_person = person.id_person
        name = person.name
        genre = person.genre
        date_of_birth = datetime.datetime.strptime(person.date_of_birth, "%d/%m/%Y").strftime("%Y-%m-%d")
        query = f"""INSERT INTO PERSON (id_person, name, genre, date_of_birth) 
                VALUES ('{person.id_person}', '{person.name}',  '{person.genre}', '{date_of_birth}')"""      
        conn.execute(query)
    print(f"All insertions successful!")
except Exception as e:        
    print(f"Insertion Failed! Error: {e}")    

All insertions successful!


In [10]:
for manufacturer in manufacturers:
    try:
        query = f"""INSERT INTO MANUFACTURER (name_manufacturer) VALUES ('{manufacturer}')"""
        conn.execute(query)
        print(f"Insertion successful!")
    except Exception as e:
        print(f"Insertion Failed! Error: {e}")   

Insertion successful!
Insertion successful!
Insertion successful!
Insertion successful!
Insertion successful!
Insertion successful!


In [11]:
df_manufacturer = pd.read_sql('manufacturer', con=conn)

In [12]:
df_manufacturer

,id_manufacturer,name_manufacturer
0,1,Fiat
1,2,Volkswagen
2,3,Lifan
3,4,Ford
4,5,Hyndai
5,6,Renault


In [13]:

dict_manufacturer = df_manufacturer.set_index(df_manufacturer.name_manufacturer)['id_manufacturer'].to_dict()
dict_manufacturer


{'Fiat': 1, 'Volkswagen': 2, 'Lifan': 3, 'Ford': 4, 'Hyndai': 5, 'Renault': 6}

In [14]:
df_models.manufacturer = df_models.manufacturer.map(dict_manufacturer) 
df_models


,model,manufacturer
0,Uno Attractive 1.0,1
1,Mobi Easy 1.5,1
2,Fox Connect 1.6,2
4,530 Talent 1.5,3
5,Ka S 1.0,4
6,Hyundai HB20 1.0 Sense,5
8,Gol 1.0,2
10,Grand Siena Attractive 1.,1
15,Kwid Life 1.0,6
27,Argo 1.0,1


In [15]:
df_models = df_models.rename(columns={'manufacturer' : 'id_manufacturer'})
df_models


,model,id_manufacturer
0,Uno Attractive 1.0,1
1,Mobi Easy 1.5,1
2,Fox Connect 1.6,2
4,530 Talent 1.5,3
5,Ka S 1.0,4
6,Hyundai HB20 1.0 Sense,5
8,Gol 1.0,2
10,Grand Siena Attractive 1.,1
15,Kwid Life 1.0,6
27,Argo 1.0,1


In [16]:
try: 
    for model in df_models.itertuples():
        name_model = model.model
        id_manufacturer = model.id_manufacturer   
        query = f"""INSERT INTO CAR_MODEL (model_description, id_manufacturer) 
                values ('{name_model}',{id_manufacturer})"""
        conn.execute(query)
    print(f"All insertions successful!")
except Exception as e:        
    print(f"Insertion Failed! Error: {e}")    


All insertions successful!
